In [ ]:
import os 
import re
import matplotlib.pyplot as plt
import numpy as np
import librosa, librosa.display

In [ ]:
data_dir = "../Datasets/aleman/"

In [ ]:
emotions = ["anger", "boredom", "disgust", "anxiety", "happiness", "sadness", "neutral"]
emotion_label = {
    "W" : "anger",
    "L" : "boredorm", 
    "E" : "disgust",
    "A" : "anxiety",
    "F" : "happiness",
    "T" : "sadness",
    "N" : "neutral"
}

In [ ]:
for emotion in emotions:
    os.mkdir(f"{data_dir}/{emotion}")

In [ ]:
def melSpectrogram(dir, file):
    signal, sr = librosa.load(f'{dir}\{file}')
    n_fft = 2048
    hop_length = 512
    mel_signal = librosa.feature.melspectrogram(y=signal, sr=sr, hop_length=hop_length, n_fft=n_fft)
    spectrogram = np.abs(mel_signal)
    power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
    plt.figure(figsize=(8, 7))
    librosa.display.specshow(power_to_db, sr=sr, x_axis="time", y_axis="mel", cmap="magma", 
     hop_length=hop_length)

    plt.axis('off')     
    emotion = file[5]
    try:
        plt.savefig(f'{data_dir}/{emotion_label[emotion]}/mel{file[:-4]}', bbox_inches='tight',pad_inches = 0)
    except:
        pass

In [ ]:
for file in os.listdir(data_dir):
    if(re.match(".*wav$", file)):
        melSpectrogram(data_dir, file)

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Resizing, Rescaling, Dropout
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
session = InteractiveSession(config=config)

In [ ]:
import tensorflow as tf
l2Reg =  tf.keras.regularizers.L2()

In [ ]:
n_class = 9
model = Sequential()

model.add(Resizing(128,157))
model.add(Rescaling(1.0/255))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=n_class,activation="softmax"))

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
import keras
from tensorflow.keras.optimizers import Adam

In [ ]:
traindata = image_dataset_from_directory("../Datasets/mezcla/", color_mode="rgb", batch_size=32,seed=123, shuffle=True, validation_split=0.2, subset="training", image_size=(224,224))
testdata = image_dataset_from_directory("../Datasets/mezcla/", color_mode="rgb", batch_size=32,seed=123, shuffle=True, validation_split=0.2, subset="validation", image_size=(224,224))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
hist = model.fit(traindata, validation_data= testdata,epochs=200, batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["val_accuracy"])
plt.show()

In [ ]:
model.save("modeloMixSinRegularizacion.h5")